In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler,  LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.linear_model import  LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import make_scorer, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, classification_report, mean_squared_error, cohen_kappa_score, roc_auc_score, mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_excel('/content/Soil_Fertility_Data.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Soil_Fertility_Data.xlsx'

In [ ]:
# df = pd.read_csv('/content/Soil_Fertility_Data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Calculate the mean of the 'N (kg/ha)' column
mean_value = df['N (kg/ha)'].mean()

# Fill missing values in the 'N (kg/ha)' column with the mean
df['N (kg/ha)'] = df['N (kg/ha)'].fillna(mean_value)

In [ ]:
# Calculate the mean of the 'Zn (ppm)' column
mean_value = df['Zn (ppm)'].mean()

# Fill missing values in the 'Zn (ppm)' column with the mean
df['Zn (ppm)'] = df['Zn (ppm)'].fillna(mean_value)

In [ ]:
# Calculate the mean of the 'Fe (ppm)' column
mean_value = df['Fe (ppm)'].mean()

# Fill missing values in the 'Fe (ppm)' column with the mean
df['Fe (ppm)'] = df['Fe (ppm)'].fillna(mean_value)

In [ ]:
# Calculate the mean of the 'Cu (ppm)' column
mean_value = df['Cu (ppm)'].mean()

# Fill missing values in the 'Cu (ppm)' column with the mean
df['Cu (ppm)'] = df['Cu (ppm)'].fillna(mean_value)

In [ ]:
# Calculate the mean of the 'Mn (ppm)' column
mean_value = df['Mn (ppm)'].mean()

# Fill missing values in the 'Mn (ppm)' column with the mean
df['Mn (ppm)'] = df['Mn (ppm)'].fillna(mean_value)

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['Location','CaCO3 (%)', 'CEC (m.e./100 g soil)'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'fruit' column
df['Soil Fertility'] = label_encoder.fit_transform(df['Soil Fertility'])

In [ ]:
df.sample(10)

In [ ]:
df.columns

In [ ]:
# Select the columns you want to include in the heatmap
columns_for_heatmap = ['pH', 'EC', 'OC (%)', 'OM (%)', 'N (kg/ha)',
                       'P (P2O5 kg.ha)', 'K (K2O kg/ha)', 'Zn (ppm)',
                       'Fe (ppm)', 'Cu (ppm)', 'Mn (ppm)','Soil Fertility']

In [ ]:
# Calculate the correlation matrix
plt.figure(figsize = (13,11))
sns.heatmap(df[columns_for_heatmap].corr() , center = 0, annot = True)
plt.show()


In [ ]:
# Define the features you want to scale/transform
features = ['pH', 'EC', 'OC (%)', 'OM (%)', 'N (kg/ha)',
                       'P (P2O5 kg.ha)', 'K (K2O kg/ha)', 'Zn (ppm)',
                       'Fe (ppm)', 'Cu (ppm)', 'Mn (ppm)','Soil Fertility']

# Separate the features for scaling
dist = df[features]

# Initialize scalers and transformer
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
power_transformer = PowerTransformer()

# Apply MinMaxScaler
dist_min_max = min_max_scaler.fit_transform(dist)
df_min_max = pd.DataFrame(dist_min_max, columns=features)

# Apply StandardScaler
dist_standard = standard_scaler.fit_transform(dist)
df_standard = pd.DataFrame(dist_standard, columns=features)

# Apply PowerTransformer
dist_power = power_transformer.fit_transform(dist)
df_power = pd.DataFrame(dist_power, columns=features)

# Create subplots: Each feature will have a set of 4 histograms (Original, MinMax, Standard, Power)
num_features = len(features)
fig, axes = plt.subplots(nrows=num_features, ncols=4, figsize=(20, 4 * num_features))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot histograms for each feature
for i, col in enumerate(features):
    # Original data
    sns.histplot(df[col], ax=axes[i * 4], kde=True)
    axes[i * 4].set_title(f'Original: {col}')
    axes[i * 4].set_xlabel('')
    axes[i * 4].set_ylabel('')

    # MinMax Scaled data
    sns.histplot(df_min_max[col], ax=axes[i * 4 + 1], kde=True)
    axes[i * 4 + 1].set_title(f'MinMax Scaled: {col}')
    axes[i * 4 + 1].set_xlabel('')
    axes[i * 4 + 1].set_ylabel('')

    # Standard Scaled data
    sns.histplot(df_standard[col], ax=axes[i * 4 + 2], kde=True)
    axes[i * 4 + 2].set_title(f'Standard Scaled: {col}')
    axes[i * 4 + 2].set_xlabel('')
    axes[i * 4 + 2].set_ylabel('')

    # Power Transformed data
    sns.histplot(df_power[col], ax=axes[i * 4 + 3], kde=True)
    axes[i * 4 + 3].set_title(f'Power Transformed: {col}')
    axes[i * 4 + 3].set_xlabel('')
    axes[i * 4 + 3].set_ylabel('')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:
sns.pairplot(df_power, hue='Soil Fertility', palette='deep')

In [ ]:
X = df.drop(columns=['Soil Fertility'])
y = df.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# Initialize PowerTransformer
pt = PowerTransformer()

# Fit and transform the training data
X_train_pt = pt.fit_transform(X_train)

# Transform the test data
X_test_pt = pt.transform(X_test)

In [ ]:
df_train_pt = pd.DataFrame(X_train_pt)

In [ ]:
# Custom scorer for specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn = cm[0, 0]
    fp = cm[0, 1]
    specificity = tn / (tn + fp)
    return specificity


# Define the scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',  # Sensitivity
    'specificity': make_scorer(specificity_score),  # Specificity
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'kappa': make_scorer(cohen_kappa_score)
}

In [ ]:
# Initialize the RandomForestClassifier with OOB score enabled
rf_classifier = RandomForestClassifier(max_samples=0.75, random_state=42, oob_score=True)

# Perform cross-validation
cv_results = cross_validate(rf_classifier, X_train_pt, y_train, cv=10, scoring=scoring)

# Calculate mean and standard deviation for each metric
rf_mean_accuracy = np.mean(cv_results['test_accuracy'])
rf_mean_precision = np.mean(cv_results['test_precision'])
rf_mean_recall = np.mean(cv_results['test_recall'])  # Sensitivity
rf_mean_specificity = np.mean(cv_results['test_specificity'])
rf_mean_f1 = np.mean(cv_results['test_f1'])
rf_mean_roc_auc = np.mean(cv_results['test_roc_auc'])
rf_mean_kappa = np.mean(cv_results['test_kappa'])

# Perform predictions using cross-validation
y_pred_prob = cross_val_predict(rf_classifier, X_train_pt, y_train, cv=10, method='predict_proba')

# Calculate RMSE
rf_rmse = np.sqrt(mean_squared_error(y_train, y_pred_prob[:, 1]))

# Calculate MAE
rf_mae = mean_absolute_error(y_train, y_pred_prob[:, 1])

# Print the results
print("Mean Accuracy:        ", rf_mean_accuracy)
print("Mean Precision:       ", rf_mean_precision)
print("Mean Recall:          ", rf_mean_recall)  # Sensitivity
print("Mean Specificity:     ", rf_mean_specificity)
print("Mean F1 Score:        ", rf_mean_f1)
print("Mean ROC AUC:         ", rf_mean_roc_auc)
print("Mean Kappa:           ", rf_mean_kappa)
print("Root Mean Squared Error (RMSE):", rf_rmse)
print("Mean Absolute Error (MAE):     ", rf_mae)

In [ ]:
# Initialize the DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Perform cross-validation
cv_results_dt = cross_validate(dt_classifier, X_train_pt, y_train, cv=10, scoring=scoring)

# Calculate mean and standard deviation for each metric
dt_mean_accuracy = np.mean(cv_results_dt['test_accuracy'])
dt_mean_precision = np.mean(cv_results_dt['test_precision'])
dt_mean_recall = np.mean(cv_results_dt['test_recall'])  # Sensitivity
dt_mean_specificity = np.mean(cv_results_dt['test_specificity'])
dt_mean_f1 = np.mean(cv_results_dt['test_f1'])
dt_mean_roc_auc = np.mean(cv_results_dt['test_roc_auc'])
dt_mean_kappa = np.mean(cv_results_dt['test_kappa'])

# Perform predictions using cross-validation
y_pred_prob_dt = cross_val_predict(dt_classifier, X_train_pt, y_train, cv=10, method='predict_proba')

# Calculate RMSE
dt_rmse = np.sqrt(mean_squared_error(y_train, y_pred_prob_dt[:, 1]))

# Calculate MAE
dt_mae = mean_absolute_error(y_train, y_pred_prob_dt[:, 1])

# Print the results
print("Decision Tree Classifier Results:")
print("Mean Accuracy:        ", dt_mean_accuracy)
print("Mean Precision:       ", dt_mean_precision)
print("Mean Recall:          ", dt_mean_recall)  # Sensitivity
print("Mean Specificity:     ", dt_mean_specificity)
print("Mean F1 Score:        ", dt_mean_f1)
print("Mean ROC AUC:         ", dt_mean_roc_auc)
print("Mean Kappa:           ", dt_mean_kappa)
print("Root Mean Squared Error (RMSE):", dt_rmse)
print("Mean Absolute Error (MAE):     ", dt_mae)

In [ ]:
# Initialize the LogisticRegression model
lr_classifier = LogisticRegression(random_state=42, max_iter=1000)

# Perform cross-validation
cv_results_lr = cross_validate(lr_classifier, X_train_pt, y_train, cv=10, scoring=scoring)

# Calculate mean and standard deviation for each metric
lr_mean_accuracy = np.mean(cv_results_lr['test_accuracy'])
lr_mean_precision = np.mean(cv_results_lr['test_precision'])
lr_mean_recall = np.mean(cv_results_lr['test_recall'])  # Sensitivity
lr_mean_specificity = np.mean(cv_results_lr['test_specificity'])
lr_mean_f1 = np.mean(cv_results_lr['test_f1'])
lr_mean_roc_auc = np.mean(cv_results_lr['test_roc_auc'])
lr_mean_kappa = np.mean(cv_results_lr['test_kappa'])

# Perform predictions using cross-validation
y_pred_prob_lr = cross_val_predict(lr_classifier, X_train_pt, y_train, cv=10, method='predict_proba')

# Calculate RMSE
lr_rmse = np.sqrt(mean_squared_error(y_train, y_pred_prob_lr[:, 1]))

# Calculate MAE
lr_mae = mean_absolute_error(y_train, y_pred_prob_lr[:, 1])

# Print the results
print("Logistic Regression Results:")
print("Mean Accuracy:        ", lr_mean_accuracy)
print("Mean Precision:       ", lr_mean_precision)
print("Mean Recall:          ", lr_mean_recall)  # Sensitivity
print("Mean Specificity:     ", lr_mean_specificity)
print("Mean F1 Score:        ", lr_mean_f1)
print("Mean ROC AUC:         ", lr_mean_roc_auc)
print("Mean Kappa:           ", lr_mean_kappa)
print("Root Mean Squared Error (RMSE):", lr_rmse)
print("Mean Absolute Error (MAE):     ", lr_mae)

In [ ]:
# Initialize the SVC model with probability enabled
svc_classifier = SVC(probability=True, random_state=42)

# Perform cross-validation
cv_results_svc = cross_validate(svc_classifier, X_train_pt, y_train, cv=10, scoring=scoring)

# Calculate mean and standard deviation for each metric
svc_mean_accuracy = np.mean(cv_results_svc['test_accuracy'])
svc_mean_precision = np.mean(cv_results_svc['test_precision'])
svc_mean_recall = np.mean(cv_results_svc['test_recall'])  # Sensitivity
svc_mean_specificity = np.mean(cv_results_svc['test_specificity'])
svc_mean_f1 = np.mean(cv_results_svc['test_f1'])
svc_mean_roc_auc = np.mean(cv_results_svc['test_roc_auc'])
svc_mean_kappa = np.mean(cv_results_svc['test_kappa'])

# Perform predictions using cross-validation
y_pred_prob_svc = cross_val_predict(svc_classifier, X_train_pt, y_train, cv=10, method='predict_proba')

# Calculate RMSE
svc_rmse = np.sqrt(mean_squared_error(y_train, y_pred_prob_svc[:, 1]))

# Calculate MAE
svc_mae = mean_absolute_error(y_train, y_pred_prob_svc[:, 1])

# Print the results
print("Support Vector Classifier Results:")
print("Mean Accuracy:        ", svc_mean_accuracy)
print("Mean Precision:       ", svc_mean_precision)
print("Mean Recall:          ", svc_mean_recall)  # Sensitivity
print("Mean Specificity:     ", svc_mean_specificity)
print("Mean F1 Score:        ", svc_mean_f1)
print("Mean ROC AUC:         ", svc_mean_roc_auc)
print("Mean Kappa:           ", svc_mean_kappa)
print("Root Mean Squared Error (RMSE):", svc_rmse)
print("Mean Absolute Error (MAE):     ", svc_mae)

In [ ]:
# Initialize the GaussianNB model
nb_classifier = GaussianNB()

# Perform cross-validation
cv_results_nb = cross_validate(nb_classifier, X_train_pt, y_train, cv=10, scoring=scoring)

# Calculate mean and standard deviation for each metric
nb_mean_accuracy = np.mean(cv_results_nb['test_accuracy'])
nb_mean_precision = np.mean(cv_results_nb['test_precision'])
nb_mean_recall = np.mean(cv_results_nb['test_recall'])  # Sensitivity
nb_mean_specificity = np.mean(cv_results_nb['test_specificity'])
nb_mean_f1 = np.mean(cv_results_nb['test_f1'])
nb_mean_roc_auc = np.mean(cv_results_nb['test_roc_auc'])
nb_mean_kappa = np.mean(cv_results_nb['test_kappa'])

# Perform predictions using cross-validation
y_pred_prob_nb = cross_val_predict(nb_classifier, X_train_pt, y_train, cv=10, method='predict_proba')

# Calculate RMSE
nb_rmse = np.sqrt(mean_squared_error(y_train, y_pred_prob_nb[:, 1]))

# Calculate MAE
nb_mae = mean_absolute_error(y_train, y_pred_prob_nb[:, 1])

# Print the results
print("Naive Bayes Classifier Results:")
print("Mean Accuracy:        ", nb_mean_accuracy)
print("Mean Precision:       ", nb_mean_precision)
print("Mean Recall:          ", nb_mean_recall)  # Sensitivity
print("Mean Specificity:     ", nb_mean_specificity)
print("Mean F1 Score:        ", nb_mean_f1)
print("Mean ROC AUC:         ", nb_mean_roc_auc)
print("Mean Kappa:           ", nb_mean_kappa)
print("Root Mean Squared Error (RMSE):", nb_rmse)
print("Mean Absolute Error (MAE):     ", nb_mae)

In [ ]:
# Initialize the KNeighborsClassifier
knn_classifier = KNeighborsClassifier()

# Perform cross-validation
cv_results_knn = cross_validate(knn_classifier, X_train_pt, y_train, cv=10, scoring=scoring)

# Calculate mean and standard deviation for each metric
knn_mean_accuracy = np.mean(cv_results_knn['test_accuracy'])
knn_mean_precision = np.mean(cv_results_knn['test_precision'])
knn_mean_recall = np.mean(cv_results_knn['test_recall'])  # Sensitivity
knn_mean_specificity = np.mean(cv_results_knn['test_specificity'])
knn_mean_f1 = np.mean(cv_results_knn['test_f1'])
knn_mean_roc_auc = np.mean(cv_results_knn['test_roc_auc'])
knn_mean_kappa = np.mean(cv_results_knn['test_kappa'])

# Perform predictions using cross-validation
y_pred_prob_knn = cross_val_predict(knn_classifier, X_train_pt, y_train, cv=10, method='predict_proba')

# Calculate RMSE
knn_rmse = np.sqrt(mean_squared_error(y_train, y_pred_prob_knn[:, 1]))

# Calculate MAE
knn_mae = mean_absolute_error(y_train, y_pred_prob_knn[:, 1])

# Print the results
print("K-Nearest Neighbors Classifier Results:")
print("Mean Accuracy:        ", knn_mean_accuracy)
print("Mean Precision:       ", knn_mean_precision)
print("Mean Recall:          ", knn_mean_recall)  # Sensitivity
print("Mean Specificity:     ", knn_mean_specificity)
print("Mean F1 Score:        ", knn_mean_f1)
print("Mean ROC AUC:         ", knn_mean_roc_auc)
print("Mean Kappa:           ", knn_mean_kappa)
print("Root Mean Squared Error (RMSE):", knn_rmse)
print("Mean Absolute Error (MAE):     ", knn_mae)


In [ ]:
accuracy = [rf_mean_accuracy*100, dt_mean_accuracy*100, lr_mean_accuracy*100, svc_mean_accuracy*100, nb_mean_accuracy*100, knn_mean_accuracy*100]
precision = [rf_mean_precision*100, dt_mean_precision*100, lr_mean_precision*100, svc_mean_precision*100, nb_mean_precision*100, knn_mean_precision*100]
recall = [rf_mean_recall*100, dt_mean_recall*100, lr_mean_recall*100, svc_mean_recall*100, nb_mean_recall*100, knn_mean_recall*100]
specificity = [rf_mean_specificity*100, dt_mean_specificity*100, lr_mean_specificity*100, svc_mean_specificity*100, nb_mean_specificity*100, knn_mean_specificity*100]
f1 = [rf_mean_f1*100, dt_mean_f1*100, lr_mean_f1*100, svc_mean_f1*100, nb_mean_f1*100, knn_mean_f1*100]
roc_auc = [rf_mean_roc_auc*100, dt_mean_roc_auc*100, lr_mean_roc_auc*100, svc_mean_roc_auc*100, nb_mean_roc_auc*100, knn_mean_roc_auc*100]
kappa = [rf_mean_kappa*100, dt_mean_kappa*100, lr_mean_kappa*100, svc_mean_kappa*100, nb_mean_kappa*100, knn_mean_kappa*100]
rmse = [rf_rmse, dt_rmse, lr_rmse, svc_rmse, nb_rmse, knn_rmse]
mae = [rf_mae, dt_mae, lr_mae, svc_mae, nb_mae, knn_mae]

In [ ]:
categories1 = ['Random Forest', 'Decision Tree', 'Logistic Regression', 'Support Vector Machine', 'Naive Bayes', 'K-Nearest Neighbour']
metrices = ['accuracy', 'precision', 'recall', 'specificity', 'f1', 'roc_auc', 'kappa', 'rmse', 'mae']
categories = ['RF', 'DT', 'Log_Reg', 'SVM', 'NB', 'KNN']

In [ ]:
# Accuracy
values = accuracy

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('Accuracy Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = precision

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('Precision Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = recall

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('Recall(Sensitivity) Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = rmse

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('RMSE Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = mae

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('MAE Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = specificity

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('Specificity Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = f1

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('F1 score Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = roc_auc

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('ROC-AUC Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()

In [ ]:
# Accuracy
values = kappa

# Create a bar plot
plt.bar(categories, values, color='skyblue')

# Add title and labels
plt.title('Kappa Bar Plot')
plt.xlabel('Categories')
plt.ylabel('Values')

# Set y-axis limits
plt.ylim(70, 100)

# Display the plot
plt.show()